In [1]:
#libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joshu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\joshu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#init.py
import json
words = []
classes = []
documents = []
ignore_words = ['?', '!']
intents = json.loads(open('intents.json').read())
kaggle_intents = json.loads(open('kaggle_intent.json').read())
c_intents = json.loads(open('complete_intents.json').read())

In [3]:
#words.py
words = []
documents = []
classes = []
#print(f'words.py {len(words)}')
# for intent in intents['intents']:
#     for pattern in intent['patterns']:

#         w = nltk.word_tokenize(pattern)
#         words.extend(w)
#         documents.append((w, intent['tag']))

#         if intent['tag'] not in classes:
#             classes.append(intent['tag'])

#This is where the kaggle_dataset is implemented. Remove this if we do not want to train with it.
for intent in c_intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [4]:
#train_init
from sklearn.model_selection import train_test_split

# initializing training data
training = []

output_empty = [0] * len(classes)
for doc in documents:
    
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    #print(len(words))
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
#train_x = list(training[:,0])
#train_y = list(training[:,1])

X = list(training[:,0])
y = list(training[:,1])
train_x, test_x, train_y, test_y = train_test_split(X,y, test_size=0.33, random_state=42)


print("Training data created")

Training data created


In [5]:
#Train.py
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Activation, Dropout

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1, validation_data=(test_x, test_y))
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
17/17 [==============================] - 0s 14ms/step - loss: 2.8175 - accuracy: 0.1429 - val_loss: 2.6331 - val_accuracy: 0.0476
Epoch 2/200
17/17 [==============================] - 0s 2ms/step - loss: 2.4438 - accuracy: 0.2381 - val_loss: 2.4982 - val_accuracy: 0.0238
Epoch 3/200
17/17 [==============================] - 0s 2ms/step - loss: 2.0336 - accuracy: 0.3690 - val_loss: 2.3795 - val_accuracy: 0.1905
Epoch 4/200
17/17 [==============================] - 0s 2ms/step - loss: 2.3436 - accuracy: 0.2619 - val_loss: 2.3340 - val_accuracy: 0.2143
Epoch 5/200
17/17 [==============================] - 0s 2ms/step - loss: 2.0364 - accuracy: 0.3929 - val_loss: 2.3792 - val_accuracy: 0.1667
Epoch 6/200
17/17 [==============================] - 0s 2ms/step - loss: 1.9909 - accuracy: 0.3690 - val_loss: 2.2984 - val_accuracy: 0.2381
Epoch 7/200
17/17 [==============================] - 0s 2ms/step - loss: 1.7693 - accuracy: 0.4048 - val_loss: 2.2221 - val_accuracy: 0.2381
Epoch 8/200
